In [47]:
"""
Creating Sentinel-2 cloud-free composites for pre-fire fuel conditions in Google Earth Engine (GEE)
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


# Calculate Sentinel-based MNDWI, EVI, and LAI for fire bounds

Bring in the fire bounds (representing bounding geometry for AFD data within each fire). Calculate the pre-fire Sentinel-2 metrics.

In [216]:
# Load the fire bounds
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/afd_aspen-fires_2018_to_2023_bounds')
fires = fires.filter(ee.Filter.eq('na_l3name','Southern Rockies'))
print(fires.first().propertyNames().getInfo())

['Fire_Year', 'Fire_ID', 'Start_Day', 'ICS_Cs_DOY', 'Fire_Name', 'ICS_Ig_DOY', 'End_Day', 'na_l3name', 'system:index']


In [218]:
# load the FRP gridded data
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats')
print(f"{grid.size().getInfo()} total gridcells.")
print(grid.first().propertyNames().getInfo())

# Get fire IDs
fire_ids = grid.aggregate_array('Fire_ID').distinct().getInfo()
fire_ids = [int(fid) for fid in fire_ids]
print(len(set(grid.aggregate_array('Fire_ID').getInfo())))

49047 total gridcells.
['Fire_Year', 'grid_index', 'Fire_ID', 'max_date', 'afd_count', 'Ig_Date', 'first_date', 'last_date', 'Last_Dat_1', 'system:index']
58


In [219]:
fires = fires.filter(ee.Filter.inList('Fire_ID', fire_ids))
print(fires.size().getInfo())

58


In [220]:
# Grab a list of Sentinel-2 bands we will need for indices
# Keep RGB for visualization of the composites
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B11']
print(f"Extracting S2 bands: {s2_bands}")

Extracting S2 bands: ['B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B11']


In [221]:
def generate_composites(geom_id):
    """ 
    Function to create a cloud-free image composite within polygon bounds
    Operates on a unique identifier (Fire ID)
    Args:
    - geom_id: the unique ID for polygon data within which to create the composite image
    """
    
    fire = fires.filter(ee.Filter.eq('Fire_ID', geom_id)).first() # get the fire bounds
    bounds = fire.geometry()
    fire_id = fire.get('Fire_ID') # get the Fire ID
    
    ig_year = ee.Number.parse(fire.get('Fire_Year')) # fire year
    start_doy = ee.Number.parse(fire.get('ICS_Ig_DOY')).subtract(14)
    end_doy = ee.Number.parse(fire.get('ICS_Cs_DOY')).subtract(1)
    
    # Filter the S2-MSI collection
    s2_filter = ee.Filter.And(
      ee.Filter.bounds(bounds), # filter to the fire bounds
      ee.Filter.calendarRange(ee.Number(ig_year), ee.Number(ig_year), 'year'), # fire year
      ee.Filter.calendarRange(start_doy, end_doy, 'DAY_OF_YEAR'), # 60 days pre-ignition
      ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60) # remove the most cloudy days
    )
    
    # Image Collection
    s2 = s2l1c.filter(s2_filter).select(s2_bands) # only keep the needed bands for indices
    
    # Grab some statistics from the collection
    n = s2.size();
    start_date = ee.Date(s2.first().get('system:time_start'));
    end_date = ee.Date(s2.limit(1,'system:time_start',False).first().get('system:time_start'));
    
    # Join to the Cloud Score Plus
    csPlus_s = csPlus.filter(
    ee.Filter.And(
      ee.Filter.bounds(bounds), # filter to the fire bounds
      ee.Filter.calendarRange(ig_year, ig_year, 'year'), # fire year
      ee.Filter.calendarRange(start_doy, end_doy, 'DAY_OF_YEAR'), # 60 days pre-ignition
    )
    ).select([QA_BAND]) # select the quality band
    
    # Mask out occluded pixels
    s2 = join_collections(s2, csPlus_s)
    s2 = s2.map(
      lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)) # > CLEAR_THRESHOLD == occluded
    ).select(s2_bands)
    
    # Calculate the spectral indices, create the median composite, clip to bounds
    s2 = s2.map(calc_indices).median().clip(bounds)
    # Select the indices and visible bands
    s2 = s2.select(['B2','B3','B4','MNDWI','EVI','LAI','NDVI705'])
    # Add Fire_ID as a property
    s2 = s2.set('Fire_ID',geom_id)
    
    return s2


def calc_indices(image):
    """ Calculates spectral indices from Sentinel-2 image data """
    
    # Modified Normalized Difference Water Index (MNDWI)
    # https://www.mdpi.com/2072-4292/8/4/354
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')

    # Enhanced Vegetation Index (EVI)
    evi = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))", {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B3')
        }).rename('EVI')

    image = image.addBands([mndwi, evi])

    # Leaf Area Index (LAI)
    lai = image.expression(
        '(3.618 * EVI - 0.118)', {
            'EVI': image.select('EVI')
        }).rename('LAI')

    # red-edge NDVI
    ndvi705 = image.normalizedDifference(['B6', 'B5']).rename('NDVI705')

    return image.addBands([lai,ndvi705])


def string_cat(item, y):
    """ Function to concatenate strings (rename bands) """
    return ee.String(item).cat(y)

def apply_string_cat(x, y):
    return x.map(lambda item: string_cat(item, y))

# Function to join collections
def join_collections(col1, col2):
    joined = ee.ImageCollection(ee.Join.saveFirst('cs').apply(
        primary=col1,
        secondary=col2,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        )
    ))
    return joined.map(lambda image: image.addBands(image.get('cs')))

print("Functions ready !")

Functions ready !


In [ ]:
# Extract the Sentinel-2 imagery for each fire.

In [223]:
# Load the S2-MSI Level 1C and cloud score plus
s2l1c = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

QA_BAND = 'cs' # cloud score band
CLEAR_THRESHOLD = 0.60 # occluded pixel threshold

composites = ee.List(fire_ids).map(lambda fire_id: generate_composites(fire_id))

img_col = ee.ImageCollection.fromImages(composites)

# Create a mosaic image for export.
mosaic = img_col.mosaic()

print("Done !")

Done !


In [224]:
# print(f"Band names: {img_col.first().bandNames().getInfo()}") # timeout on full dataset
print(mosaic.bandNames().getInfo()) # timeout on full dataset

['B2', 'B3', 'B4', 'MNDWI', 'EVI', 'LAI', 'NDVI705']


In [225]:
print(f"Number of images: {img_col.size().getInfo()}")

Number of images: 58


In [240]:
# Initialize a map (timeout on full dataset)
Map = geemap.Map()
Map.addLayerControl()

# Vis params
vis_params_lai = {
    'bands': ['LAI'],
    'min': -50,
    'max': 100,
}
vis_params_mndwi = {
    'bands': ['MNDWI'],
    'min': -1,
    'max': 1,
}
vis_params_ndvi705 = {
    'bands': ['NDVI705'],
    'min': -1,
    'max': 1,
}
vis_params_rgb = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Add the layers to the map
Map.addLayer(mosaic, vis_params_lai, "LAI")
Map.addLayer(mosaic, vis_params_mndwi, "MNDWI")
Map.addLayer(mosaic, vis_params_ndvi705, "NDVI705")
Map.addLayer(mosaic, vis_params_rgb, "RGB")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [241]:
# # Export the mosaic imagery.
# bands_ = ['MNDWI','LAI','NDVI705']
# mosaic = mosaic.select(bands_)

# task = ee.batch.Export.image.toAsset(
#     image=mosaic,  
#     description='viirs_snpp_jpss1_aspenfires_S2MSI_mosaic', 
#     assetId='projects/jfsp-aspen/assets/viirs_snpp_jpss1_aspenfires_S2MSI_mosaic', 
#     region=fires.geometry().bounds(), 
#     scale=30, 
#     crs="EPSG:4326", 
#     maxPixels=1e13
# )

# # Start the export task
# task.start()
# print("Export to Earth Engine Asset started!")
# # Monitor the task until it's finished
# monitor_export(task, 300) 

In [ ]:
# summarize Sentinel-2 metrics within forest types from the USFS TreeMap

In [242]:
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['ALSTK', 'BALIVE', 'CANOPYPCT', 'CARBON_D', 'CARBON_DWN', 'CARBON_L', 'DRYBIO_D', 'DRYBIO_L', 'FLDSZCD', 'FLDTYPCD', 'FORTYPCD', 'GSSTK', 'QMD_RMRS', 'SDIPCT_RMRS', 'STANDHT', 'STDSZCD', 'TPA_DEAD', 'TPA_LIVE', 'Value', 'VOLBFNET_L', 'VOLCFNET_D', 'VOLCFNET_L']


In [243]:
# grab the metrics we care about
treemap = treemap.select(['FORTYPCD','BALIVE','SDIPCT_RMRS','STANDHT','TPA_LIVE','TPA_DEAD'])
treemap.first().bandNames().getInfo()

['FORTYPCD', 'BALIVE', 'SDIPCT_RMRS', 'STANDHT', 'TPA_LIVE', 'TPA_DEAD']

In [250]:
def summarize_s2(image):
    """ Reductions on S2 metrics by fire """
    fire_id = ee.String(image.get('Fire_ID'))
    grid_fire = grid.filter(ee.Filter.eq('Fire_ID', fire_id))
    grid_fire = grid_fire.select(['grid_index'])
    
    # calculate the reductions within FRP gridcells for each forest type
    def species_metrics(ftr):
        
        """Calculate Sentinel-2 metrics for each species type within a grid cell."""
        # Mosaic the TreeMap data (if needed)
        treemap_image = treemap.mosaic()
        
        # Get the species histogram for the grid cell
        hist = treemap_image.select('FORTYPCD').reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=ftr.geometry(),
            scale=30,
            maxPixels=1e13
        ).get('FORTYPCD')
        
        hist_dict = ee.Dictionary(hist)
    
        # Function to calculate metrics for each species type
        def add_species_metrics(key, current_dict):
            key_str = ee.String(key)
            
            # Mask the Sentinel-2 image by the species type
            masked_image = image.updateMask(treemap_image.select('FORTYPCD').eq(ee.Number.parse(key)))
            
            # Calculate mean metrics for the masked image
            metrics = masked_image.select(['MNDWI', 'LAI', 'NDVI705']).reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=ftr.geometry(),
                scale=30,
                maxPixels=1e13
            )
            
            # Construct a sub-dictionary for this species
            sp_metrics = ee.Dictionary.fromLists(
                ee.List(['MNDWI', 'LAI', 'NDVI705']).map(lambda col: key_str.cat('_').cat(col)),
                ee.List(['MNDWI', 'LAI', 'NDVI705']).map(lambda col: metrics.get(col))
            )
            
            # Combine this species' metrics with the current dictionary
            return ee.Dictionary(current_dict).combine(sp_metrics)
    
        metrics_dict = hist_dict.keys().iterate(add_species_metrics, ee.Dictionary())
        
        # Flatten the dictionary into separate properties
        metrics_flat = ee.Dictionary(metrics_dict).map(
            lambda key, value: ee.Algorithms.String(value)  # Convert numbers to strings for CSV compatibility
        )
        
        return ftr.set(metrics_flat)
        
    # Map species_metrics over the grid cells
    species_stats = grid_fire.map(species_metrics)
    
    return species_stats

# map the function
spp_stats = img_col.map(summarize_s2)
spp_stats_fc = ee.FeatureCollection(spp_stats.flatten())
print("Number of features in stats:", spp_stats_fc.size().getInfo())

Number of features in stats: 49047


In [251]:
sample = spp_stats_fc.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df.head()

,182_LAI,182_MNDWI,182_NDVI705,185_LAI,185_MNDWI,185_NDVI705,221_LAI,221_MNDWI,221_NDVI705,225_LAI,...,999_NDVI705,224_LAI,224_MNDWI,224_NDVI705,266_LAI,266_MNDWI,266_NDVI705,974_LAI,974_MNDWI,974_NDVI705
0,25.617594119335127,-0.4054753309344762,0.2161390946845138,7.110853554233482,-0.30285869751657757,0.10131493955850601,19.520277973132977,-0.439436218078993,0.23579941618687084,23.25329507051684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24.032243986133572,-0.41057705177533294,0.25223059643849993,NaN,NaN,NaN,18.829253360670428,-0.395479919465608,0.25160553085105813,29.611960593730796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13.606796366044666,-0.44497536344014343,0.21818947366091868,13.284000521038205,-0.4486459042350848,0.2253560510101315,21.97228303075572,-0.4615681001069053,0.25480538358056176,25.875123142708112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13.878555871460211,-0.41449576191338544,0.15793267688513027,7.2459176170305435,-0.37619756550392236,0.12687609820434842,15.981291124178048,-0.4157717422893641,0.17320791771466204,16.569807253307015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14.440263022702778,-0.4039846061822737,0.21273641106427696,9.893019360101613,-0.4191067637974849,0.14432598639636626,21.176356769058792,-0.36964899211204244,0.23128050537834668,23.021043639039558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [252]:
df.columns

Index(['182_LAI', '182_MNDWI', '182_NDVI705', '185_LAI', '185_MNDWI',
       '185_NDVI705', '221_LAI', '221_MNDWI', '221_NDVI705', '225_LAI',
       '225_MNDWI', '225_NDVI705', '369_LAI', '369_MNDWI', '369_NDVI705',
       '371_LAI', '371_MNDWI', '371_NDVI705', '901_LAI', '901_MNDWI',
       '901_NDVI705', '971_LAI', '971_MNDWI', '971_NDVI705', 'grid_index',
       '922_LAI', '922_MNDWI', '922_NDVI705', '184_LAI', '184_MNDWI',
       '184_NDVI705', '201_LAI', '201_MNDWI', '201_NDVI705', '704_LAI',
       '704_MNDWI', '704_NDVI705', '706_LAI', '706_MNDWI', '706_NDVI705',
       '281_LAI', '281_MNDWI', '281_NDVI705', '703_LAI', '703_MNDWI',
       '703_NDVI705', '261_LAI', '261_MNDWI', '261_NDVI705', '267_LAI',
       '267_MNDWI', '267_NDVI705', '268_LAI', '268_MNDWI', '268_NDVI705',
       '999_LAI', '999_MNDWI', '999_NDVI705', '224_LAI', '224_MNDWI',
       '224_NDVI705', '266_LAI', '266_MNDWI', '266_NDVI705', '974_LAI',
       '974_MNDWI', '974_NDVI705'],
      dtype='object')

In [ ]:
# export the table as a CSV.
export_task = ee.batch.Export.table.toDrive(
    collection=spp_stats_fc,
    description='gridstats_fortypcd_s2',
    fileNamePrefix='gridstats_fortypcd_s2',
    fileFormat='CSV', 
    folder='TreeMap'
)
export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
monitor_export(export_task, 120)

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo